CICFlowMeter is an IP based method and unfortunately no information about the Ethernet address is available. We use the Ethernet Source address (MAC address) to tag the Device data. In this context, this code allows us to establish the relationship between the source MAC address and the source IP address. So we can label the CICFlowMeter data.

In [35]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
import os
import shutil

In [36]:
# Find pcap files

In [37]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

path="./iot-data"
pcaps=find_the_way(path,".pcap")
pcaps

['./iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-26_17_23_20.68s.pcap',
 './iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-26_18_38_57.65s.pcap',
 './iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-26_19_39_01.66s.pcap',
 './iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_14_32_13.65s.pcap',
 './iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_14_51_27.65s.pcap',
 './iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_15_13_35.66s.pcap',
 './iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_15_35_48.64s.pcap',
 './iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_15_57_55.65s.pcap',
 './iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_16_20_02.66s.pcap',
 './iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_16_42_11.66s.pcap',
 './iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_17_04_20.65s.pcap',
 './iot-data\\uk\\allure-speaker

In [38]:
len(pcaps)

37744

In [43]:
def folder(f_name): #this function creates a folder named "attacks" in the program directory.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

## MERGING PCAPS

At this stage, we will utilize the CICFlowMeter tool to extract features from pcap files. However, we encountered an issue due to the large number of individual pcap files, totaling 37744. Unfortunately, the tool does not handle such high volumes well and generates errors.
For instance, when attempting to extract features from a subset consisting of only 8023 normal UK files, we encountered the following issues 

#### ERROR LOGS

```python 
"""2024-04-14 16:48:58 - Error while opening file for capture: C:\Users\kahra\pcap_proje\CICFlowMeterv3\bin\logs\smarter-coffee-mach@google_on@2019-05-04_19_54_22.23s.pcap: Too many open files"""

"""2024-04-14 16:49:24 - Error while opening file for capture: C:\Users\kahra\pcap_proje\CICFlowMeterv3\bin\logs\magichome-strip@android_lan_on@2019-04-28_21_42_09.25s.pcap: Too many open files"""

"""2024-04-14 16:49:55 - Error while opening file for capture: C:\Users\kahra\pcap_proje\CICFlowMeterv3\bin\logs\magichome-strip@alexa_on@2019-04-28_19_12_16.23s.pcap: Too many open files"""
```
---------------------------------------------------------------------------------------------

To address this challenge, we opted to consolidate all files associated with a single state of a device into a unified pcap file. For example, in the directory "iot-data\uk\allure-speaker\power," we merged three files—2019-04-26_12_23_35.224s.pcap, 2019-04-26_12_41_35.204s.pcap, and 2019-04-26_12_59_02.249s.pcap—into a single file named "iot-data\uk\allure-speaker\power.pcap."





In [39]:
essiz=[]
for p in pcaps:
    temp=p.replace("\\","/")
    cut=temp.rfind("/")
    essiz.append(temp[:cut+1])
essiz=list(set(essiz))

In [40]:
for i in tqdm(essiz):
    cut=i[:-1].rfind("/")
    temp=i[cut+1:-1]
    files_add=find_the_way(i,'.pcap')
    files_add=str(files_add)
    files_add=files_add.replace("[","")
    files_add=files_add.replace("]","")
    files_add=files_add.replace("\'","")
    files_add=files_add.replace(",","")
    #print(files_add)
    command=f"mergecap -w {i[:i.find(temp)]}/{temp}.pcap {files_add}"
    os.system(command)
    shutil.rmtree(i)
    #print(command)

100%|██████████████████████████████████████████████████████████████████████████████| 1349/1349 [02:38<00:00,  8.53it/s]


In [41]:
folders=[]
for p in pcaps:
    newname=p.replace("\\","/")
    newname=newname.split("/")
    if newname[2] not in folders:
        folders.append(newname[2])
folders

['uk', 'uk-vpn', 'us', 'us-vpn']

In [44]:
for f in folders:
    print(f)
    folder(f"./csvs/{f}")
    address=f"{path}/{f}/"
    pcaps=find_the_way(address,".pcap")
    folder(address.replace("./iot-data","./pcaps"))
    for p in tqdm(pcaps):
        newname=p.replace("\\","/")
        #newname=newname.replace("./iot-data","./pcaps")
        newname=newname[len(address):].replace("/","@")
        newname=f"{address}{newname}"
        #print(p,"\n",newname)
        os.rename(p,newname)
        


uk


100%|██████████████████████████████████████████████████████████████████████████████| 264/264 [00:00<00:00, 2723.04it/s]


uk-vpn


100%|██████████████████████████████████████████████████████████████████████████████| 264/264 [00:00<00:00, 1940.08it/s]


us


100%|██████████████████████████████████████████████████████████████████████████████| 412/412 [00:00<00:00, 2258.54it/s]


us-vpn


100%|██████████████████████████████████████████████████████████████████████████████| 409/409 [00:00<00:00, 1794.54it/s]


In [45]:
def list_and_delete_empty_folders(root_folder):
    for folder_path, subfolders, files in os.walk(root_folder, topdown=False):
        for subfolder in subfolders:
            subfolder_path = os.path.join(folder_path, subfolder)
            if not os.listdir(subfolder_path):
                print(f"{subfolder_path} is empty, deleting...")
                os.rmdir(subfolder_path)
            else:
                print(f"{subfolder_path} is not empty, not deleting.")

list_and_delete_empty_folders(path)

./iot-data\uk\allure-speaker is empty, deleting...
./iot-data\uk\appletv is empty, deleting...
./iot-data\uk\blink-camera is empty, deleting...
./iot-data\uk\blink-security-hub is empty, deleting...
./iot-data\uk\bosiwo-camera-wired is empty, deleting...
./iot-data\uk\charger-camera is empty, deleting...
./iot-data\uk\dlink-camera is empty, deleting...
./iot-data\uk\echodot is empty, deleting...
./iot-data\uk\echoplus is empty, deleting...
./iot-data\uk\echospot is empty, deleting...
./iot-data\uk\firetv is empty, deleting...
./iot-data\uk\google-home is empty, deleting...
./iot-data\uk\google-home-mini is empty, deleting...
./iot-data\uk\honeywell-thermostat is empty, deleting...
./iot-data\uk\insteon-hub is empty, deleting...
./iot-data\uk\lightify-hub is empty, deleting...
./iot-data\uk\magichome-strip is empty, deleting...
./iot-data\uk\nest-tstat is empty, deleting...
./iot-data\uk\netatmo-weather-station is empty, deleting...
./iot-data\uk\ring-doorbell is empty, deleting...
./io

In [46]:
# MAC and IP features extraction from Pcaps
pcaps=find_the_way(path,".pcap")

In [47]:
for p in tqdm(pcaps):
    name=p.replace("\\","/")

    #LAbelling
    output=name.replace(".pcap",".csv")  
    if " " not in p:
        command=f"tshark -r {p}  -T fields -e eth.src -e ip.src   -E header=y -E separator=, -E quote=d -E occurrence=f >{output}"
    else:
        command=f"tshark -r \"{p}\" -T fields  -e eth.src  -e ip.src  -E header=y -E separator=, -E quote=d -E occurrence=f >\"{output}\""
    os.system(command)

100%|██████████████████████████████████████████████████████████████████████████████| 1349/1349 [14:12<00:00,  1.58it/s]


In [48]:
a=find_the_way(path,".csv")
a

['./iot-data\\uk\\allure-speaker@android_lan_audio_off.csv',
 './iot-data\\uk\\allure-speaker@android_lan_audio_on.csv',
 './iot-data\\uk\\allure-speaker@android_wan_audio_off.csv',
 './iot-data\\uk\\allure-speaker@android_wan_audio_on.csv',
 './iot-data\\uk\\allure-speaker@power.csv',
 './iot-data\\uk\\allure-speaker@voice.csv',
 './iot-data\\uk\\allure-speaker@volume.csv',
 './iot-data\\uk\\appletv@local_menu.csv',
 './iot-data\\uk\\appletv@local_voice.csv',
 './iot-data\\uk\\appletv@power.csv',
 './iot-data\\uk\\blink-camera@alexa_stop.csv',
 './iot-data\\uk\\blink-camera@alexa_watch.csv',
 './iot-data\\uk\\blink-camera@android_lan_photo.csv',
 './iot-data\\uk\\blink-camera@android_lan_watch.csv',
 './iot-data\\uk\\blink-camera@android_wan_photo.csv',
 './iot-data\\uk\\blink-camera@android_wan_watch.csv',
 './iot-data\\uk\\blink-camera@local_move.csv',
 './iot-data\\uk\\blink-camera@power.csv',
 './iot-data\\uk\\blink-security-hub@alexa_stop.csv',
 './iot-data\\uk\\blink-security-hu

In [49]:
name_list=[]
for i in a:
    name_list.append(i.replace("\\","/"))

In [50]:
# SELECTED DEVICES
macs={'18:b4:30:c8:d8:28': 'nest-tstat',
 '18:74:2e:41:4d:35': 'echodot',
 '50:c7:bf:5a:2e:a0': 'tplink-plug',
 '84:c0:ef:2f:42:cc': 'samsungtv-wired',
 '24:fd:5b:04:1b:75': 'smartthings-hub',
 '98:84:e3:e4:35:bd': 'ring-doorbell',
 '00:0e:f3:3b:85:e5': 'insteon-hub',
 '88:de:a9:08:03:b9': 'roku-tv',
 '00:17:88:68:5f:61': 't-philips-hub',
 '14:91:82:b4:4b:5f': 't-wemo-plug',
 '50:c7:bf:a0:f3:76': 'tplink-bulb',
 '00:71:47:c0:91:93': 'echospot',
 'dc:4f:22:c1:58:05': 'magichome-strip',
 '84:18:26:7d:cf:a2': 'lightify-hub',
 '20:df:b9:5f:41:7e': 'google-home-mini',
 '34:ce:00:83:99:35': 'xiaomi-hub',
 '6c:56:97:35:39:f4': 'firetv',
 '08:66:98:a2:21:9e': 'appletv',
 'fc:a1:83:38:e0:2d': 'echoplus',
 'b0:ce:18:27:9f:e4': 'sengled-hub',
 'b0:d5:9d:b9:f0:b4': 'yi-camera',
 '78:a5:dd:1a:15:19': 'wansview-cam-wired',
 '00:03:7f:4f:c6:b5': 'blink-security-hub',
 'f4:b8:5e:31:73:db': 'blink-camera',
 'dc:4f:22:28:b6:5b': 'sousvide',
 'fc:03:9f:93:22:62': 'samsungtv-wired',
 'd0:52:a8:a4:e6:46': 'smartthings-hub',
 '20:df:b9:13:e5:2e': 'google-home-mini',
 '78:a5:dd:28:a1:b7': 'wansview-cam-wired',
 '0c:8c:24:0b:be:fb': 'yi-camera',
 '00:03:7f:96:d8:ec': 'blink-security-hub',
 'f4:b8:5e:68:8f:35': 'blink-camera',
 '58:ef:68:99:7d:ed': 't-wemo-plug',
 '00:0e:f3:2c:d4:04': 'insteon-hub',
 'ec:b5:fa:00:98:da': 't-philips-hub',
 'c8:3a:6b:fa:1c:00': 'roku-tv',
 '84:18:26:7c:1a:56': 'lightify-hub',
 '50:c7:bf:ca:3f:9d': 'tplink-bulb',
 'b0:ce:18:20:43:bf': 'sengled-hub',
 '68:c6:3a:ba:c2:6b': 'sousvide',
 'cc:f7:35:49:f4:05': 'echodot',
 '5c:41:5a:29:ad:97': 'echospot',
 '00:fc:8b:84:22:10': 'echoplus',
 '50:c7:bf:b1:d2:78': 'tplink-plug',
 'cc:f7:35:25:af:4d': 'firetv',
 'f0:45:da:36:e6:23': 'ring-doorbell',
 'dc:4f:22:89:fc:e7': 'magichome-strip',
 '7c:49:eb:88:da:82': 'xiaomi-hub',
 '50:32:37:b8:c7:0f': 'appletv',
 '64:16:66:2a:98:62': 'nest-tstat'}

In [51]:
# Associates IP addresses and MAC addresses within CSV files.

In [52]:
bosh={}
for ii in name_list:
    temp={}
    df=pd.read_csv(ii)
    for i in df.values:
        if (i[0] not in temp) and (i[0] in macs):
            temp[i[0]]=[]
        elif i[0] in temp:
            if ("." in str(i[1])):
                if (i[1] not in temp[i[0]])  :
                    temp[i[0]].append(i[1])
    bosh[ii]=temp
bosh

{'./iot-data/uk/allure-speaker@android_lan_audio_off.csv': {'d0:52:a8:a4:e6:46': []},
 './iot-data/uk/allure-speaker@android_lan_audio_on.csv': {'d0:52:a8:a4:e6:46': []},
 './iot-data/uk/allure-speaker@android_wan_audio_off.csv': {'d0:52:a8:a4:e6:46': []},
 './iot-data/uk/allure-speaker@android_wan_audio_on.csv': {'d0:52:a8:a4:e6:46': []},
 './iot-data/uk/allure-speaker@power.csv': {'fc:03:9f:93:22:62': ['192.168.20.223'],
  'd0:52:a8:a4:e6:46': ['192.168.20.120']},
 './iot-data/uk/allure-speaker@voice.csv': {},
 './iot-data/uk/allure-speaker@volume.csv': {'d0:52:a8:a4:e6:46': []},
 './iot-data/uk/appletv@local_menu.csv': {'50:32:37:b8:c7:0f': ['192.168.20.133']},
 './iot-data/uk/appletv@local_voice.csv': {'50:32:37:b8:c7:0f': ['192.168.20.133']},
 './iot-data/uk/appletv@power.csv': {'50:32:37:b8:c7:0f': ['192.168.20.133',
   '0.0.0.0']},
 './iot-data/uk/blink-camera@alexa_stop.csv': {'f4:b8:5e:68:8f:35': ['0.0.0.0',
   '192.168.20.138']},
 './iot-data/uk/blink-camera@alexa_watch.csv':

In [57]:
Device_Namer={}
for  i in bosh:
    #print()
    #print(i)
    essiz=[]
    for ii in bosh[i]:
        #print(macs[ii],bosh[i][ii])
        #print(ii)
        for iii in bosh[i][ii]:
            #print(iii)
            if iii not in essiz:
                essiz.append(iii)
            else:print(iii)
Device_Namer

{}

In [60]:
Device_Namer={}
for  i in bosh:
    temp={}
    for ii in bosh[i]:
        for iii in bosh[i][ii]:
            temp[iii]=ii
            print(temp)

    a=i.replace("/","-")
    a=a[2:-4]
    Device_Namer[a]=temp

{'192.168.20.223': 'fc:03:9f:93:22:62'}
{'192.168.20.223': 'fc:03:9f:93:22:62', '192.168.20.120': 'd0:52:a8:a4:e6:46'}
{'192.168.20.133': '50:32:37:b8:c7:0f'}
{'192.168.20.133': '50:32:37:b8:c7:0f'}
{'192.168.20.133': '50:32:37:b8:c7:0f'}
{'192.168.20.133': '50:32:37:b8:c7:0f', '0.0.0.0': '50:32:37:b8:c7:0f'}
{'0.0.0.0': 'f4:b8:5e:68:8f:35'}
{'0.0.0.0': 'f4:b8:5e:68:8f:35', '192.168.20.138': 'f4:b8:5e:68:8f:35'}
{'0.0.0.0': 'f4:b8:5e:68:8f:35'}
{'0.0.0.0': 'f4:b8:5e:68:8f:35', '192.168.20.138': 'f4:b8:5e:68:8f:35'}
{'0.0.0.0': 'f4:b8:5e:68:8f:35'}
{'0.0.0.0': 'f4:b8:5e:68:8f:35', '192.168.20.138': 'f4:b8:5e:68:8f:35'}
{'0.0.0.0': 'f4:b8:5e:68:8f:35'}
{'0.0.0.0': 'f4:b8:5e:68:8f:35', '192.168.20.138': 'f4:b8:5e:68:8f:35'}
{'0.0.0.0': 'f4:b8:5e:68:8f:35'}
{'0.0.0.0': 'f4:b8:5e:68:8f:35', '192.168.20.138': 'f4:b8:5e:68:8f:35'}
{'0.0.0.0': 'f4:b8:5e:68:8f:35'}
{'0.0.0.0': 'f4:b8:5e:68:8f:35', '192.168.20.138': 'f4:b8:5e:68:8f:35'}
{'0.0.0.0': 'f4:b8:5e:68:8f:35'}
{'0.0.0.0': 'f4:b8:5e:68:8

In [61]:
Device_Namer

{'iot-data-uk-allure-speaker@android_lan_audio_off': {},
 'iot-data-uk-allure-speaker@android_lan_audio_on': {},
 'iot-data-uk-allure-speaker@android_wan_audio_off': {},
 'iot-data-uk-allure-speaker@android_wan_audio_on': {},
 'iot-data-uk-allure-speaker@power': {'192.168.20.223': 'fc:03:9f:93:22:62',
  '192.168.20.120': 'd0:52:a8:a4:e6:46'},
 'iot-data-uk-allure-speaker@voice': {},
 'iot-data-uk-allure-speaker@volume': {},
 'iot-data-uk-appletv@local_menu': {'192.168.20.133': '50:32:37:b8:c7:0f'},
 'iot-data-uk-appletv@local_voice': {'192.168.20.133': '50:32:37:b8:c7:0f'},
 'iot-data-uk-appletv@power': {'192.168.20.133': '50:32:37:b8:c7:0f',
  '0.0.0.0': '50:32:37:b8:c7:0f'},
 'iot-data-uk-blink-camera@alexa_stop': {'0.0.0.0': 'f4:b8:5e:68:8f:35',
  '192.168.20.138': 'f4:b8:5e:68:8f:35'},
 'iot-data-uk-blink-camera@alexa_watch': {'0.0.0.0': 'f4:b8:5e:68:8f:35',
  '192.168.20.138': 'f4:b8:5e:68:8f:35'},
 'iot-data-uk-blink-camera@android_lan_photo': {'0.0.0.0': 'f4:b8:5e:68:8f:35',
  '

### IP MAC address relationships are stored to be used in feature extraction.

In [62]:
import pickle
with open('Device_Namer.pickle', 'wb') as handle:
    pickle.dump(Device_Namer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('Device_Namer.pickle', 'rb') as handle:
    b = pickle.load(handle)

print(Device_Namer == b)

True
